# Aide Jazmín González Cruz

In [ ]:
# https://pypi.org/project/sodapy/
# https://www.sustainabilist.com/blog/chicago-data-analysis-a-internship-project
# https://dev.socrata.com/consumers/getting-started.html

# https://holowczak.com/getting-started-with-nyc-opendata-and-the-socrata-api/5/

# https://dev.socrata.com/docs/functions/between.html
# https://pydigger.com/pypi/sodapy

# https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5/data

# https://data.cityofchicago.org/resource/4ijn-s7e5.csv?$where=inspection_date>'2021-01-01' limit=2000

## Cargando librerías

In [4]:
import pandas as pd
from sodapy import Socrata
import pickle
from datetime import date, timedelta

2021-02-16
2021-02-09


In [4]:
import boto3
# saber qué versión de Boto estamos ocupando
print(boto3.__version__)

1.17.9


# API Chicago

In [ ]:
date_object = date.today()
print(date_object)

dt = date.today() - timedelta(7)
print(dt)

In [5]:
# The Host Name for the API endpoint (the https:// part will be added automatically)
data_url='data.cityofchicago.org'   
# The data set at the API endpoint (311 data in this case)
data_set='4ijn-s7e5'   
# The app token created in the prior steps
app_token='QcC2iXBnip0eXLBnriENWyee4'   
# Create the client to point to the API endpoint
client = Socrata(data_url,app_token)      
# Set the timeout to 60 seconds    
client.timeout = 60
# Retrieve the first 2000 results returned as JSON object from the API
# The SoDaPy library converts this JSON object to a Python list of dictionaries
results = client.get(data_set, limit=300000)
# Convert the list of dictionaries to a Pandas data frame
df = pd.DataFrame.from_records(results)
df.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,2484780,RED STAR LIQUORS,RED STAR LIQUORS,2432531,Grocery Store,Risk 2 (Medium),2725-2727 N MILWAUKEE AVE,CHICAGO,IL,60647,2021-02-11T00:00:00.000,Complaint Re-Inspection,Pass,57. ALL FOOD EMPLOYEES HAVE FOOD HANDLER TRAIN...,41.93078708970233,-87.70998753402094,"{'latitude': '-87.70998753402094', 'longitude'..."
1,2484768,MARZ COMMINITY BREWING CO.,MARZ COMMUNITY BREWING CO.,2770949,Liquor,Risk 3 (Low),1950 N WESTERN AVE,CHICAGO,IL,60647,2021-02-11T00:00:00.000,License,Pass,"53. TOILET FACILITIES: PROPERLY CONSTRUCTED, S...",41.917134801370274,-87.68754385201615,"{'latitude': '-87.68754385201615', 'longitude'..."
2,2484766,LA FORTALEZA DE ARCELIA GUERRERO,LA FORTALEZA DE ARCELIA GUERRERO,2575370,Restaurant,Risk 1 (High),5958 W DIVERSEY AVE,CHICAGO,IL,60639,2021-02-11T00:00:00.000,Complaint Re-Inspection,Pass w/ Conditions,37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER ...,41.931249818095836,-87.7759069982422,"{'latitude': '-87.7759069982422', 'longitude':..."
3,2484759,Jimmy Johns,Jimmy Johns,1898470,Restaurant,Risk 1 (High),51 W DIVISION ST,CHICAGO,IL,60610,2021-02-11T00:00:00.000,Canvass Re-Inspection,Pass,NaN,41.903806014540685,-87.63058113117621,"{'latitude': '-87.63058113117621', 'longitude'..."
4,2484775,CAFE TOLA #4,CAFE TOLA,2627325,Restaurant,Risk 1 (High),2489 N MILWAUKEE AVE,CHICAGO,IL,60647,2021-02-11T00:00:00.000,Canvass Re-Inspection,Pass,NaN,41.92663502691596,-87.70319925259288,"{'latitude': '-87.70319925259288', 'longitude'..."


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216269 entries, 0 to 216268
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   inspection_id    216269 non-null  object
 1   dba_name         216269 non-null  object
 2   aka_name         213784 non-null  object
 3   license_         216252 non-null  object
 4   facility_type    211382 non-null  object
 5   risk             216200 non-null  object
 6   address          216269 non-null  object
 7   city             216109 non-null  object
 8   state            216219 non-null  object
 9   zip              216224 non-null  object
 10  inspection_date  216269 non-null  object
 11  inspection_type  216268 non-null  object
 12  results          216269 non-null  object
 13  violations       158485 non-null  object
 14  latitude         215553 non-null  object
 15  longitude        215553 non-null  object
 16  location         215553 non-null  object
dtypes: object(

In [7]:
# Save the data frame to a CSV file
df.to_csv("output/2020-02-02_d.csv")
pickle.dump(df, open("output/2020-02-02_.pkl", "wb"))

In [14]:
#data_url="https://data.cityofchicago.org/resource/4ijn-s7e5.csv?$where=inspection_date>'2020-01-01'"

# The Host Name for the API endpoint (the https:// part will be added automatically)
data_url='data.cityofchicago.org'   
# The data set at the API endpoint (311 data in this case)
data_set="4ijn-s7e5"
# Create the client to point to the API endpoint
client = Socrata(data_url,app_token)      
# Set the timeout to 60 seconds    
client.timeout = 60
# Retrieve the first 2000 results returned as JSON object from the API
# The SoDaPy library converts this JSON object to a Python list of dictionaries
results2 = client.get(data_set, where="inspection_date > '2021-01-01' ", limit=2000)
# Convert the list of dictionaries to a Pandas data frame
df2 = pd.DataFrame.from_records(results2)
df2.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,2472253,CHITOWN SANDWICH CLUB,CHITOWN SANDWICH CLUB,2574541,Restaurant,Risk 1 (High),2921 N CLARK ST,CHICAGO,IL,60657,2021-01-04T00:00:00.000,Canvass,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.935452917475686,-87.64686341693728,"{'latitude': '-87.64686341693728', 'longitude'..."
1,2472237,AFC SUSHI @ JEWEL OSCO#2502,AFC SUSHI @ JEWEL OSCO,1985397,Restaurant,Risk 1 (High),2550 N CLYBOURN,CHICAGO,IL,60614,2021-01-04T00:00:00.000,Canvass,Pass w/ Conditions,56. ADEQUATE VENTILATION & LIGHTING; DESIGNATE...,41.92787172311833,-87.67325881463049,"{'latitude': '-87.67325881463049', 'longitude'..."
2,2464207,MACHINE,MACHINE,2595542,Restaurant,Risk 1 (High),1836-1846 W DIVISION ST,CHICAGO,IL,60622,2021-01-04T00:00:00.000,Non-Inspection,No Entry,NaN,41.90335448822961,-87.67381134493849,"{'latitude': '-87.67381134493849', 'longitude'..."
3,2464210,FOURTH CHURCH DAY SCHOOL,FOURTH CHURCH DAY SCHOOL,2215767,DAYCARE,Risk 1 (High),126 E CHESTNUT ST,CHICAGO,IL,60611,2021-01-04T00:00:00.000,Canvass,Pass,NaN,41.89848890902573,-87.62455288409814,"{'latitude': '-87.62455288409814', 'longitude'..."
4,2402212,Zaytune Mediterranean Grill,Zaytune,2009543,Restaurant,Risk 1 (High),3129 S Morgan ST,CHICAGO,IL,60608,2021-01-04T00:00:00.000,Canvass,Pass w/ Conditions,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.83713294787266,-87.65097503508115,"{'latitude': '-87.65097503508115', 'longitude'..."


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1413 entries, 0 to 1412
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   inspection_id    1413 non-null   object
 1   dba_name         1413 non-null   object
 2   aka_name         1409 non-null   object
 3   license_         1413 non-null   object
 4   facility_type    1408 non-null   object
 5   risk             1413 non-null   object
 6   address          1413 non-null   object
 7   city             1408 non-null   object
 8   state            1413 non-null   object
 9   zip              1412 non-null   object
 10  inspection_date  1413 non-null   object
 11  inspection_type  1413 non-null   object
 12  results          1413 non-null   object
 13  violations       993 non-null    object
 14  latitude         1406 non-null   object
 15  longitude        1406 non-null   object
 16  location         1406 non-null   object
dtypes: object(17)
memory usage: 187.8

In [16]:
# Save the data frame to a CSV file
df2.to_csv("output/2021-01-01_d2.csv")
pickle.dump(df2, open("output/2021-01-01_d2.pkl", "wb"))

In [21]:
fecha = dt.strftime("%Y-%m-%d")
where_cond = "inspection_date > '" + fecha + "' "
where_cond

"inspection_date > '2021-02-09' "

In [22]:
results3 = client.get(data_set, where=where_cond, limit=2000)
# Convert the list of dictionaries to a Pandas data frame
df3 = pd.DataFrame.from_records(results3)
df3.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,2484753,SUBWAY,SUBWAY,1985307,Restaurant,Risk 1 (High),4242 S WENTWORTH AVE,CHICAGO,IL,60609,2021-02-10T00:00:00.000,Canvass Re-Inspection,Pass,41.81698453270526,-87.6313645580596,"{'latitude': '-87.6313645580596', 'longitude':...",NaN
1,2484705,KOKO'S MEDITERRANEAN GRILL,KOKO'S MEDITERRANEAN GRILL,2293013,Restaurant,Risk 1 (High),1760 W CHICAGO AVE,CHICAGO,IL,60622,2021-02-10T00:00:00.000,Canvass,Fail,41.89610267284695,-87.67211594992528,"{'latitude': '-87.67211594992528', 'longitude'...",60. PREVIOUS CORE VIOLATION CORRECTED - Commen...
2,2484700,SUPERMERCADO EL TAXQUENITO,SUPERMERCADO EL TAXQUENITO,2240202,Grocery Store,Risk 1 (High),6309 W GRAND AVE,CHICAGO,IL,60639,2021-02-10T00:00:00.000,Non-Inspection,No Entry,41.92189422993173,-87.78339752005009,"{'latitude': '-87.78339752005009', 'longitude'...",NaN
3,2484743,ZANNO'S FISH & CHICKEN,ZANNO'S,2704295,Restaurant,Risk 1 (High),1521 S PULASKI RD,CHICAGO,IL,60623,2021-02-10T00:00:00.000,Non-Inspection,No Entry,41.86003909876977,-87.72484229344634,"{'latitude': '-87.72484229344634', 'longitude'...",NaN
4,2484741,CLUB NUTRICION LIA,CLUB NUTRICION LIA,2663532,Restaurant,Risk 2 (Medium),3720 W 26TH ST,CHICAGO,IL,60623,2021-02-10T00:00:00.000,Non-Inspection,No Entry,41.84444353922239,-87.71821191334412,"{'latitude': '-87.71821191334412', 'longitude'...",NaN


In [23]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   inspection_id    27 non-null     object
 1   dba_name         27 non-null     object
 2   aka_name         27 non-null     object
 3   license_         27 non-null     object
 4   facility_type    27 non-null     object
 5   risk             27 non-null     object
 6   address          27 non-null     object
 7   city             27 non-null     object
 8   state            27 non-null     object
 9   zip              27 non-null     object
 10  inspection_date  27 non-null     object
 11  inspection_type  27 non-null     object
 12  results          27 non-null     object
 13  latitude         27 non-null     object
 14  longitude        27 non-null     object
 15  location         27 non-null     object
 16  violations       13 non-null     object
dtypes: object(17)
memory usage: 3.7+ KB


In [24]:
# Save the data frame to a CSV file
df3.to_csv("output/" + fecha + ".csv")
pickle.dump(df3, open("output/" + fecha + ".pkl", "wb"))

# AWS

In [1]:
import yaml

In [2]:
# este archivo yaml vive en tu conf/local/credentials.yaml
# este va en src/utils general.py
def read_yaml_file(yaml_file):
    """ load yaml cofigurations """

    config = None
    try: 
        with open (yaml_file, 'r') as f:
            config = yaml.safe_load(f)
    except:
        raise FileNotFoundError('Couldnt load the file')
    
    return config

# este va en src/utils general.py
def get_s3_credentials(credentials_file):
    credentials = read_yaml_file(credentials_file)
    s3_creds = credentials['s3']

    return s3_creds

In [6]:
#s3_creds = get_s3_credentials("../../conf/local/credentials.yaml")
s3_creds = get_s3_credentials("credentials.yaml")

session = boto3.Session(
    aws_access_key_id=s3_creds['aws_access_key_id'],
    aws_secret_access_key=s3_creds['aws_secret_access_key']
)
s3 = session.client('s3')

***Listado de solo una carpeta en específico***

In [7]:
bucket = "mapreduce1ajgc"
key = "output"

In [8]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
objects

[{'Key': 'output/_SUCCESS',
  'LastModified': datetime.datetime(2021, 2, 10, 4, 59, 36, tzinfo=tzutc()),
  'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
  'Size': 0,
  'StorageClass': 'STANDARD'},
 {'Key': 'output/part-00000',
  'LastModified': datetime.datetime(2021, 2, 10, 4, 59, 35, tzinfo=tzutc()),
  'ETag': '"05e70ed4809d6ad9cdfb18fe285e02f5"',
  'Size': 51054,
  'StorageClass': 'STANDARD'},
 {'Key': 'output/part-00001',
  'LastModified': datetime.datetime(2021, 2, 10, 4, 59, 35, tzinfo=tzutc()),
  'ETag': '"9a69a9d437d12c51ba4be21f18b9f212"',
  'Size': 51176,
  'StorageClass': 'STANDARD'},
 {'Key': 'output/part-00002',
  'LastModified': datetime.datetime(2021, 2, 10, 4, 59, 34, tzinfo=tzutc()),
  'ETag': '"afa66a14b9c2c3d85a2b5cbc4187b2f3"',
  'Size': 50955,
  'StorageClass': 'STANDARD'}]

### Creación de un bucket

In [ ]:
#crear un bucket
#bucket_name = 'test-aws-boto-clase-dpa'

#s3.create_bucket(Bucket=bucket_name, 
#                 CreateBucketConfiguration={'LocationConstraint': 'us-west-2'},
#                 ACL='private')

### Guardar archivos en el bucket

In [ ]:
# este archivo yaml vive en tu conf/local/credentials.yaml
# este va en src/utils general.py
def read_yaml_file(yaml_file):
    """ 
    load yaml cofigurations 
    """

    config = None
    try: 
        with open (yaml_file, 'r') as f:
            config = yaml.safe_load(f)
    except:
        raise FileNotFoundError('Couldnt load the file')
    
    return config

# este va en src/utils general.py
def get_s3_credentials(credentials_file):
    """ 
    Regresa credenciales de conexión a AWS 
    """
    credentials = read_yaml_file(credentials_file)
    s3_creds = credentials['s3']

    return s3_creds



def get_client(token):
    """ 
    Esta función regresa un cliente que se puede conectar a la API de inspecciones 
    de establecimiento dándole un token previamente generado.
    """
    #s3_creds = get_s3_credentials("../../conf/local/credentials.yaml")
    
    #token 
    s3_creds = get_s3_credentials("credentials.yaml")

    session = boto3.Session(
        aws_access_key_id=s3_creds['aws_access_key_id'],
        aws_secret_access_key=s3_creds['aws_secret_access_key']
    )
    
    s3_client = session.client('s3')
    
    return s3_client  


def ingesta_inicial(s3_client, limit):
    """ 
    Esta función recibe como parámetros el cliente con el que nos podemos comunicar con la API, 
    y el límite de registros que queremos obtener al llamar a la API. 
    Regresa una lista de los elementos que la API regresó.
    """
    # The Host Name for the API endpoint (the https:// part will be added automatically)
    data_url='data.cityofchicago.org'   
    # The data set at the API endpoint (311 data in this case)
    data_set='4ijn-s7e5'   
    # The app token created in the prior steps
    app_token='QcC2iXBnip0eXLBnriENWyee4'   
    # Create the client to point to the API endpoint
    client = Socrata(data_url,app_token)      
    # Set the timeout to 60 seconds    
    client.timeout = 60
    # Retrieve the first 2000 results returned as JSON object from the API
    # The SoDaPy library converts this JSON object to a Python list of dictionaries
    results = client.get(data_set, limit=300000)
    return results


def get_s3_resource():     
    """ 
    Esta función regresa un resource de S3 para poder guardar datos 
    en el bucket (checar script de aws_boto_s3).
    """
    
    #crear un bucket
    bucket_name = 'test-aws-boto-clase-dpa'

    # acceder al bucket
    # este es con el resource 
    my_bucket = s3.Bucket(bucket_name)
    
    return my_bucket
    
def guardar_ingesta(my_bucket, ruta, datos):      
    """ 
    Esta función recibe como parámetros el nombre de tu bucket de S3, 
    la ruta en el bucket en donde se guardarán los datos y los datos ingestados en csv.
    """
    
    bucket = "metodos-gran-escala"
    key = "northwind"
    
    # subiremos un archivo csv de ejemplo: pollution con ~43k registros 
    #con observaciones de contaminación del aire en una ciudad de China. 
    file_to_upload = '/home/silil/Documents/itam/mineria_datos_licenciatura/data/beijing_pollution/pollution.csv'
    file_name = file_to_upload.split(sep='/')[-1]

    # accedemos a client desde el resource
    #s3.meta.client.upload_file(file_to_upload, bucket_name, file_name)
    
    # si están accediendo a través de client es asi
    s3.upload_file(file_to_upload, bucket_name, file_name)